In [ ]:
def SOS(g,xi0,tmin,tmax,exact,solvers,tvec = [0.1, 0.5, 1.0], plot=True): # Second Order Solver
    solutionvals = []
    texact = np.arange(tmin,tmax,0.1)
    exactvals = exact(texact,xi0[0]) # this is for this particular exact solution
    
    for dt in tvec:
        numvals = math.ceil(abs((tmax-tmin)/dt))++1
        
        numcols = 1 + 8 # number of columns assuming 2nd order, y1 and y2 for each solver + 1 shared vector of times.
        # We want this structure to be fixed: 0 is time, 1-2 are expEuler data, 3-4 are impEuler data, 5-6 are RK2 data, 7-8 are RK4 data. 
        # If any solvers aren't used, their allocated space remains zeros.
        # We do this because dynamically adjusting which data to reference based on what solvers there are and the number of solvers there are is difficult
        vals = np.zeros((numvals, numcols)) 
        vals [:,0] = np.linspace(tmin,tmax,numvals)
        truedt = vals[1,0] - vals[0,0]
        print('True step size t=',truedt)

        numsolvers = len(solvers)
        for solverindex in np.arange(0,numsolvers): 
            solvefunc = solvers[solverindex]
            if solvefunc.__name__ == 'expEuler':
                # print('expEuler')
                storageindex = 0
            elif solvefunc.__name__ == 'impEuler':
                # print('impEuler')
                storageindex = 1
            elif solvefunc.__name__ == 'RK2':
                # print('RK2')
                storageindex = 2
            else: # RK4
                # print('RK4/catch')
                storageindex = 3
            vals[0,2*storageindex+1] = xi0[0] #initialize the solution matrix
            vals[0,2*storageindex+2] = xi0[1]
            
            
            for i in np.arange(0,numvals-1):
                tempstatevec = np.array([vals[i,1],vals[i,2]]) # Note that the temporary state vector is y1;y2
                tval = vals[i,0] # 0 is tvals
                yvec = solvefunc(g,dt,tempstatevec,tval) # solve the actual ODE
                vals[i+1,2*storageindex+1] = yvec[0] # store the values
                vals[i+1,2*storageindex+2] = yvec[1]
        # I now recognize I should have defined a single solver function, allowed it the option to use any given solver, then spit out only one list of t's and y's.
        # I also recognize I should define my exact solution function separately.
        # print(vals)
        if plot:
            plot_vals(vals[:,0],vals[:,1],vals[:,3],vals[:,5],vals[:,7],texact,exactvals,logscale = False)
        # solutionvals.append(vals)
    return [solutionvals,texact,exactvals] #I recognize that this is a janky way to output the data, and plan to fix it for any future ODE solving.



In [ ]:
test = SOS(g3b,y0,0,6,exact3b,solvers=[expEuler,impEuler,RK4],tvec = [0.15, 0.5, 1.0], plot=False)

In [ ]:
test = SOS(g3a,y0,0,6,exact3a,solvers=[expEuler,impEuler,RK4],tvec = [0.15, 0.5, 1.0], plot=False)